In [29]:
import torch
import os
import json
import re
from collections import defaultdict
import pickle
from torch import nn

In [30]:
def read_json(filename):
    #f就表示整个文件内容
    rst=[]#存储整个文件的诗
    with open(filename,"rb") as f:
        datas=json.load(f)
        for data in datas:#datas是一个列表，每个元素是一个字典，包含一首诗
            paragraphs=data.get("paragraphs")#['行行西至一荒陂，因笑唐公不見機。', '莫惜驌驦輸令尹，漢東宮闕早時歸。']
            all_data=""#存储每一部诗
            for words in paragraphs:#遍历诗的每一句
                all_data+=words
            if all_data !="":
                #all_data=sentenceParse(all_data)
                rst.append(all_data)
    return rst
            #all_data=sentenceParse(all_data)
            #print(all_data)
        
    

In [31]:
datas=[]#所有的唐诗
file_path="./chinese-poetry/"
#获取所有的唐诗文件
for filename in os.listdir(file_path):
    if filename.startswith("poet.tang"):
        rst=read_json(file_path+filename)
        #print(len(rst))
        datas.extend(rst)
w2i={}

for words in datas:
    for word in words:
        if word not in w2i:
            w2i[word]=len(w2i)

w2i['<EOP>'] = len(w2i)
w2i['<START>'] = len(w2i)
VOCAB_SIZE = len(w2i)
print(VOCAB_SIZE)
f=open("w2i.txt","wb")
f.write(pickle.dumps(w2i))
f.close()#关闭文件，刷新缓冲区的数据
#给每一诗后面中添加上"<EOP>",而且现在的data[i]为一个列表，列表中每一个元素是一个字
for i in range(len(datas)):
    datas[i] = list(datas[i])
    datas[i].append("<EOP>")

 

3504


In [32]:
class Language_Module(nn.Module):
    def __init__(self,nwords,emb_size,hidden_size):
        super(Language_Module,self).__init__()
        self.emb=nn.Embedding(nwords,emb_size)
        self.rnn=nn.LSTM(emb_size,hidden_size)
        self.out=nn.Linear(hidden_size,nwords)
    def forward(self,input):
        embed=self.emb(input)
        embed=embed.unsqueeze(1)
        rnn_out,_=self.rnn(embed)
        h,b,w=rnn_out.shape
        out=rnn_out.view(-1,w)
        out=self.out(out)
        return out

In [33]:
#取出字典中键所对应的值
def make_one_hot_vec_target(word, w2i):
    rst = torch.LongTensor([w2i[word]])
    return rst
def makeForOneCase(sentence, one_hot_var_target):
    tmpIn = []
    tmpOut = []
    #print(s)
    #遍历整首诗，从整首诗的第二个开始
    for i in range(1, len(sentence)):
        word_out = sentence[i]
        word_in = sentence[i - 1]
        tmpIn.append(one_hot_var_target[word_in])
        tmpOut.append(one_hot_var_target[word_out])
    return torch.cat(tmpIn), torch.cat(tmpOut)

In [34]:
model=Language_Module(len(w2i),256,128)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
one_hot_var_target = {}
for w in w2i:
    one_hot_var_target.setdefault(w, make_one_hot_vec_target(w, w2i))
    #setdefault的意思是说如果字典中没有w则设置w的值为make_one_hot_vec_target(w, w2i)


In [35]:
epochNum = 1#epoch的次数

TRAINSIZE = len(datas)#唐诗有多少首
batch = 200
for epoch in range(epochNum):
    model.train()
    train_loss = 0
    train_counts = 0
    for case in range(TRAINSIZE):
        s = datas[case]
        t, o = makeForOneCase(s, one_hot_var_target)
        #print(t)
        #print(o)
        output = model(t)
        loss=criterion(output, o)
        train_loss += loss
        train_counts += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if case%3==0:
            print (epoch, (train_loss.float()/train_counts).item())
    model.eval()
    dev_loss = 0
    dev_counts = 0
    for case in range(TRAINSIZE):
        s = datas[case]
        t, o = makeForOneCase(s, one_hot_var_target)
        output = model(t)
        dev_loss += criterion(output, o)
        dev_counts += 1
    print (epoch,(dev_loss.float()/dev_counts).item())
    
torch.save(model,'poetry-gen.pt')

0
0 8.179521560668945
0 7.077409267425537


In [43]:
#加载模型
model = torch.load('poetry-gen.pt')
#生成古诗的最大长度为100
max_length = 100
#读取字典
w2i = open('w2i.txt', 'rb')
#加载字典
word_to_ix = pickle.load(w2i)
#生成i2w，就是将w2i反转
def i2w(w2i):
    return dict((v, k) for k, v in w2i.items())
ix_to_word = i2w(word_to_ix)

def sample(startWord='<START>'):
    #将开始词数字化
    input = make_one_hot_vec_target(startWord, word_to_ix)
    #记录生成的每一个词，从<START>开始
    output_name = "";
    if (startWord != "<START>"):
        output_name = startWord
    for i in range(max_length):
        output = model(input)#输入为一个字
        #topk为查找最大的数字topk和对应索引topi
        topv, topi = output.data.topk(1)
        #对应的索引，也就是字典中对应的那个索引
        topi = topi[0][0]
        #将索引转换为word
        w = ix_to_word[topi.item()]
        #只要w不是结束符就不结束
        if w == "<EOP>":
            break
        else:
            output_name += w
        input = make_one_hot_vec_target(w, word_to_ix)
    return output_name







print (sample())